ELM-PSO

In [28]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [39]:
# ------------------- Import Libraries -------------------
import numpy as np
import pandas as pd
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load training and testing data
df_tr = pd.read_excel('/content/gdrive/MyDrive/Web and Data/Code/9. Hybrid_ELMs/PROJECT.xlsx', sheet_name="TR")
df_ts = pd.read_excel('/content/gdrive/MyDrive/Web and Data/Code/9. Hybrid_ELMs/PROJECT.xlsx', sheet_name="TS")

data_tr = df_tr.values
data_ts = df_ts.values

X_train = data_tr[:, :-1]
Y_train = data_tr[:, -1].reshape(-1, 1)
X_test = data_ts[:, :-1]
Y_test = data_ts[:, -1].reshape(-1, 1)

# ------------------- Normalize Data -------------------
x_scaler = StandardScaler()
X_train = x_scaler.fit_transform(X_train)
X_test = x_scaler.transform(X_test)

y_scaler = StandardScaler()
Y_train = y_scaler.fit_transform(Y_train)
Y_test = y_scaler.transform(Y_test)

# Save scalers
save_dir = '/content/gdrive/MyDrive/Web and Data/Code/9. Hybrid_ELMs'
os.makedirs(save_dir, exist_ok=True)
joblib.dump(x_scaler, os.path.join(save_dir, 'x_scaler.joblib'))
joblib.dump(y_scaler, os.path.join(save_dir, 'y_scaler.joblib'))

# ------------------- ELM Model -------------------
class ELM:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.W = np.random.randn(input_dim, hidden_dim)
        self.b = np.random.randn(hidden_dim)

    def hidden_output(self, X):
        H = np.dot(X, self.W) + self.b
        return np.tanh(H)

    def predict(self, X, beta):
        H = self.hidden_output(X)
        return np.dot(H, beta)

    def get_weights_dim(self):
        return self.hidden_dim * self.output_dim

# ------------------- PSO Optimizer for ELM -------------------
class PSO_ELM:
    def __init__(self, elm, X_train, y_train, X_test, y_test, y_scaler,
                 n_particles=50, max_iter=100, w=0.4, c1=1.0, c2=2.0):
        self.elm = elm
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.y_scaler = y_scaler
        self.n_particles = n_particles
        self.max_iter = max_iter
        self.dim = elm.get_weights_dim()

        self.positions = np.random.randn(n_particles, self.dim)
        self.velocities = np.zeros((n_particles, self.dim))
        self.pbest = self.positions.copy()
        self.pbest_scores = np.array([self.evaluate(p) for p in self.positions])
        self.gbest = self.pbest[np.argmin(self.pbest_scores)]
        self.gbest_score = np.min(self.pbest_scores)

        self.w = w
        self.c1 = c1
        self.c2 = c2

    def evaluate(self, beta_flat):
        beta = beta_flat.reshape(self.elm.hidden_dim, self.elm.output_dim)
        y_pred = self.elm.predict(self.X_train, beta)
        return mean_squared_error(self.y_train, y_pred)

    def compute_metrics(self, beta_flat, X, y):
        beta = beta_flat.reshape(self.elm.hidden_dim, self.elm.output_dim)
        y_pred_scaled = self.elm.predict(X, beta)
        y_pred = self.y_scaler.inverse_transform(y_pred_scaled)
        y_true = self.y_scaler.inverse_transform(y)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        r2 = r2_score(y_true, y_pred)
        return rmse, r2

    def optimize(self):
        print(f"{'Iter':<6}{'Train RMSE':<12}{'Train R2':<10}{'Test RMSE':<12}{'Test R2':<10}")
        for i in range(self.max_iter):
            for j in range(self.n_particles):
                r1, r2 = np.random.rand(), np.random.rand()
                self.velocities[j] = (self.w * self.velocities[j] +
                                      self.c1 * r1 * (self.pbest[j] - self.positions[j]) +
                                      self.c2 * r2 * (self.gbest - self.positions[j]))
                self.positions[j] += self.velocities[j]

                score = self.evaluate(self.positions[j])
                if score < self.pbest_scores[j]:
                    self.pbest[j] = self.positions[j]
                    self.pbest_scores[j] = score
                    if score < self.gbest_score:
                        self.gbest = self.positions[j]
                        self.gbest_score = score

            train_rmse, train_r2 = self.compute_metrics(self.gbest, self.X_train, self.y_train)
            test_rmse, test_r2 = self.compute_metrics(self.gbest, self.X_test, self.y_test)
            print(f"{i+1:<6}{train_rmse:<12.4f}{train_r2:<10.4f}{test_rmse:<12.4f}{test_r2:<10.4f}")

        return self.gbest

# ------------------- Run ELM + PSO -------------------
input_dim = X_train.shape[1]
output_dim = 1
hidden_dim = 5
elm = ELM(input_dim, hidden_dim, output_dim)
pso = PSO_ELM(elm, X_train, Y_train, X_test, Y_test, y_scaler,
              n_particles=50, max_iter=100)
best_beta_flat = pso.optimize()
best_beta = best_beta_flat.reshape(hidden_dim, output_dim)

# ------------------- Final Evaluation -------------------
final_preds_scaled = elm.predict(X_test, best_beta)
final_preds = y_scaler.inverse_transform(final_preds_scaled)
Y_test_orig = y_scaler.inverse_transform(Y_test)

final_rmse = np.sqrt(mean_squared_error(Y_test_orig, final_preds))
final_r2 = r2_score(Y_test_orig, final_preds)

print(f"\n✅ Final Test RMSE: {final_rmse:.4f}")
print(f"✅ Final Test R²: {final_r2:.4f}")

# ------------------- Save Model -------------------
model_data = {
    'beta': best_beta,
    'W': elm.W,
    'b': elm.b,
    'input_dim': input_dim,
    'hidden_dim': hidden_dim,
    'output_dim': output_dim
}
model_path = os.path.join(save_dir, 'elm_pso_best_model.joblib')
joblib.dump(model_data, model_path)
print(f"✅ Model saved to: {model_path}")


Iter  Train RMSE  Train R2  Test RMSE   Test R2   
1     16.8959     0.2944    11.8576     0.3829    
2     19.3278     0.0766    8.7343      0.6652    
3     20.2252     -0.0111   8.5875      0.6764    
4     16.8940     0.2945    11.2997     0.4396    
5     16.7197     0.3090    11.2232     0.4472    
6     17.9915     0.1999    9.9669      0.5640    
7     18.3398     0.1686    9.9063      0.5693    
8     17.7230     0.2236    10.0473     0.5570    
9     16.6331     0.3161    11.9212     0.3763    
10    17.6368     0.2311    14.8619     0.0306    
11    17.7702     0.2194    15.1072     -0.0016   
12    16.5964     0.3192    11.4704     0.4226    
13    16.5361     0.3241    11.3289     0.4367    
14    16.4628     0.3301    10.5360     0.5128    
15    16.4264     0.3330    9.9133      0.5687    
16    16.4214     0.3334    10.0502     0.5567    
17    16.4161     0.3339    10.1405     0.5487    
18    16.4027     0.3350    10.2342     0.5403    
19    16.3973     0.3354    10.